In [ ]:
#imporing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import re

In [1]:
#Loading the dataset
df=pd.read_excel("Online Retail.xlsx")
df.head()

NameError: name 'pd' is not defined

In [ ]:
df.info()

In [ ]:
# Statistical summary of the dataset
df.describe()

In [ ]:
#Checking for missing values
df.isnull().sum()

In [ ]:
df["CustomerID"].value_counts()

In [ ]:
df[df['CustomerID'].isnull()].head()


In [ ]:
df[df["Quantity"]<0].head()

In [ ]:
df[df["UnitPrice"]<0].head()

In [ ]:
df["InvoiceNo"].dtype

In [ ]:
#df[df["InvoiceNo"].str.match("\D",na=False)].head()
df[df["InvoiceNo"].str.match("^\\d{6}$") == False]

In [ ]:
df[df["StockCode"].str.match("^\d{5}$") == False]

In [ ]:
df[df["StockCode"].str.match("^.$", na=False) | df["StockCode"].str.match("^.{4}$", na=False)]['StockCode'].value_counts()

In [ ]:
df.shape

In [ ]:
df['InvoiceNo']=df['InvoiceNo'].astype(str)



In [ ]:
df["InvoiceNo"] = df["InvoiceNo"].apply(lambda x: re.sub(r"\D", "", x)) 
df1 = df[df["InvoiceNo"].str.match(r"^\d{6}$", na=False)]

In [ ]:
df1.shape

In [ ]:
df1[df1["StockCode"].str.match("^\d{5}$") == False]

In [ ]:
df1["StockCode"]=df1["StockCode"].astype(str)
df1['StockCode']=df1['StockCode'].apply(lambda x: re.sub(r"\D",'',x))
df1.shape

In [ ]:
df2=df1[df1["StockCode"].str.match("^\d{5}$") == True]

In [ ]:
df2.shape

In [ ]:
df2.info()

In [ ]:
df2.CustomerID.isnull().sum()

In [ ]:
df2.dropna(subset=['CustomerID'],inplace=True)

In [ ]:
df2.shape

In [ ]:
df2.head()

In [ ]:
df2["Description"].value_counts()


In [ ]:
df2["Description"].unique()

In [ ]:
df2.describe()

In [ ]:
df2=df2[df2["Quantity"] >= 0]

In [ ]:
df2.describe()

In [ ]:
df_zero=df2[df2["UnitPrice"]==0]
df_zero

In [ ]:
df2.shape

In [ ]:
df2=df2[df2["UnitPrice"]>0.0]

In [ ]:
df2["UnitPrice"].eq(0).sum()

In [ ]:
df2.shape

In [ ]:
df2.describe()

In [ ]:
#Apply now feature engineering
df2['Total Amount']=df2["Quantity"]*df2["UnitPrice"]

In [ ]:
df2.head(10)

In [ ]:
df2["InvoiceDate"] = pd.to_datetime(df2["InvoiceDate"])  # Ensure correct datetime format
df2.set_index("InvoiceDate")["Quantity"].resample("M").sum().plot(kind="line", figsize=(10,5), title="Monthly Sales Trend")


In [ ]:
top_products = df2.groupby("StockCode")["Quantity"].sum().nlargest(10)
top_products.plot(kind="bar", figsize=(10,5), title="Top 10 Best-Selling Products")


## Monetary Value, Frequency, and Recency Matter? (RFM Analysis)

In [ ]:
latest_purchase=df2["InvoiceDate"].max()

In [ ]:
print(latest_purchase)

In [ ]:
df_rfm=df2.groupby("CustomerID").agg({
    "InvoiceDate" : lambda x: (latest_purchase - x.max()).days,
    "InvoiceNo" : "count",
    "Total Amount" : "sum"
   })

In [ ]:
df_rfm.rename(columns={"InvoiceDate" : "Recency",
                       "InvoiceNo" : "Frequency",
                       "Total Amount" : "Monetary"
                       }, inplace=True)

In [ ]:
df_rfm.head()

In [ ]:
import seaborn as sns
sns.scatterplot(x="Recency", y="Monetary", data=df_rfm, hue="Recency", palette="viridis")
plt.show()



In [ ]:
sns.histplot(df_rfm['Recency'], kde=True, color='blue', bins=20)  # kde=True adds a Kernel Density Estimate curve
plt.title('Distribution of Recency')
plt.xlabel('Recency (Days)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Apply StandardScaler to scale the data
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
df_rfm_scaled=sc.fit_transform(df_rfm)

In [ ]:
df_rfm_scaled=pd.DataFrame(df_rfm_scaled,columns=df_rfm.columns)
df_rfm_scaled.head()

In [ ]:
# Elbow Method
from sklearn.cluster import KMeans
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(df_rfm_scaled)
    wcss.append(kmeans.inertia_)
plt.plot(range(1, 11), wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
#Apply KMeans Clustering
kmeans = KMeans(n_clusters=4, init='k-means++', random_state=42)

df_rfm_scaled["Cluster"]=kmeans.fit_predict(df_rfm_scaled)

df_rfm_scaled

In [ ]:
# Get the centers of each cluster
cluster_centers = kmeans.cluster_centers_
print("Cluster Centers:\n", cluster_centers)

# Get cluster distribution (How many customers in each cluster?)
print(df_rfm_scaled['Cluster'].value_counts())


In [ ]:

# Plot Recency vs Frequency with color for each cluster
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Recency', y='Frequency', data=df_rfm_scaled, hue='Cluster', palette='viridis', s=100, alpha=0.6)
plt.title('Customer Segmentation: Recency vs Frequency')
plt.xlabel('Recency (Days)')
plt.ylabel('Frequency (Number of Purchases)')
plt.legend(title="Cluster")
plt.show()


In [ ]:

plt.scatter(df_rfm_scaled[df_rfm_scaled["Cluster"] == 0]["Recency"], 
            df_rfm_scaled[df_rfm_scaled["Cluster"] == 0]["Frequency"], 
            s=100, c='red', label='Cluster 1')

plt.scatter(df_rfm_scaled[df_rfm_scaled["Cluster"] == 1]["Recency"], 
            df_rfm_scaled[df_rfm_scaled["Cluster"] == 1]["Frequency"], 
            s=100, c='blue', label='Cluster 2')

plt.scatter(df_rfm_scaled[df_rfm_scaled["Cluster"] == 2]["Recency"], 
            df_rfm_scaled[df_rfm_scaled["Cluster"] == 2]["Frequency"], 
            s=100, c='green', label='Cluster 3')

plt.scatter(df_rfm_scaled[df_rfm_scaled["Cluster"] == 3]["Recency"], 
            df_rfm_scaled[df_rfm_scaled["Cluster"] == 3]["Frequency"], 
            s=100, c='cyan', label='Cluster 4')



plt.scatter(kmeans.cluster_centers_[:, 0], 
            kmeans.cluster_centers_[:, 1], 
            s=300, c='yellow', label='Centroids')

plt.title('Clusters of customers')
plt.xlabel('Recency')
plt.ylabel('Frequency')
plt.legend()
plt.show()

